In [ ]:
#countvectorizer-LDA and calculation of perplexity remove NaN #module 5.0
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.manifold import TSNE
import numpy as np
import pandas as pd
import spacy

def lda_create(df, name, vec, i): #df = raw data (Pandas DataFrame), name = word name list, vec = Bag of words matrix #i = the number of topics
    lda = LatentDirichletAllocation(n_components=i,random_state=1)#lda
    lda.fit(vec)
    perp = lda.perplexity(vec)
    print("topic:{}, perplexity:{}".format(i,perp))
    lda_value = lda.fit_transform(vec)#probability of each topic
    lda_index = np.argmax(lda_value,axis=1)#maximum of the probability of each group = component
    df_lda_index = pd.DataFrame(lda_index,index=df["PMID"],columns =["topic"])#dic_corpus.keys()=PMID
    df_period = df[["PMID","period"]].set_index(["PMID"])
    df_lda = pd.concat([df_lda_index,df_period],axis=1)
    display(df_lda)

    #topic words
    print("topic words top-10")
    topic_words = {}
    weight_words = {}
    n_top_words = 10
    for topic, comp in enumerate(lda.components_):
        word_idx = np.argsort(comp)[::-1][:n_top_words]
        topic_words[topic] = [name[j] for j in word_idx]
        weight_words[topic] = np.sort(comp)[::-1][:n_top_words]
    for topic, words in topic_words.items():
        print('Topic: %d' % topic)
        print('  %s' % ', '.join(words))

    #period-component
    dic_period = {}
    for j in range(i):
        ls_component = []
        for period in range(1,5):
            ls_component.append(len(df_lda[(df_lda["period"]==period)&(df_lda["topic"]==j)]))
        dic_period[j]=ls_component

    df_period = pd.DataFrame(dic_period.values(),index = dic_period.keys(),columns=["period I","period II","period III","period IV"])
    df_topic = pd.DataFrame(topic_words.values(),index = topic_words.keys(),columns=["word{}".format(i) for i in range(n_top_words)])
    df_weight = pd.DataFrame(weight_words.values(),index = weight_words.keys(),columns=["weight{}".format(i)for i in range(n_top_words)])
    df_period_topic = pd.concat([df_topic,df_weight,df_period],axis=1)
    display(df_period_topic)

    return perp, df_lda, df_period_topic


df1 = pd.read_csv("../Files/Figure3/Table7.csv")
df1["period"]=1
df2 = pd.read_csv("../Files/Figure3/Table8.csv")
df2["period"]=2
df3 = pd.read_csv("../Files/Figure3/Table9.csv")
df3["period"]=3
df4 = pd.read_csv("../Files/Figure3/Table10.csv")
df4["period"]=4
df = pd.concat([df1,df2,df3,df4],axis=0)
df.drop_duplicates(subset="PMID",inplace=True)
df.dropna(subset=["abstract"], inplace=True)#ここがmodule 5.0と違う
df.reset_index(inplace=True,drop=True)
display(df)


nlp = spacy.load("en_core_sci_lg")
with open("../Files/SupplementalTables/SupplementalTableS2.txt", "r", encoding='utf-8') as f:
    stopwords = [w.strip() for w in f]
print("=====\n stopwords\n")
print(stopwords)

dic_corpus={}
for raw in df[["PMID","abstract"]].itertuples():
    text =""
    text_cleaned=""
    doc = nlp(str(raw[2]))
    for ent in doc.ents:
        text = text+" "+str(ent).lower()
        ls = text.split()
    for i in ls:
        if i not in stopwords:
            text_cleaned = text_cleaned+" "+i
    print(raw[1])
    print(text_cleaned)
    dic_corpus[raw[1]]=text_cleaned

corpus = [text_cleaned for text_cleaned in dic_corpus.values()]
#1 countvectorizer
vectorizer = CountVectorizer()
count = vectorizer.fit_transform(corpus)
count_name=vectorizer.get_feature_names_out()

count_vec= count.toarray()
count_pd = pd.DataFrame(count_vec,index = dic_corpus.keys(),columns=count_name)
print("\n=====\ntifdf_pandas\n=====\n")
display(count_pd)

#2 TF-IDF
vectorizer = TfidfVectorizer()
tfidf = vectorizer.fit_transform(corpus)
tfidf_name=vectorizer.get_feature_names_out()

tfidf_vec= tfidf.toarray()
tfidf_pd = pd.DataFrame(tfidf_vec,index=dic_corpus.keys(),columns=tfidf_name)
print("\n=====\ntifdf_pandas\n=====\n")
display(tfidf_pd)

#3 t-SNE
print("t-SNE")
tsne = TSNE(n_components=2, random_state=0)#componentsで分類の数
W=tsne.fit_transform(count_vec)
W_tfidf=tsne.fit_transform(tfidf_vec)

#4 LDA (without TF-IDF) and n_topic optimization
perplexity = {}
for i in range(10,21):
    perp, df_lda, df_period_topic = lda_create(df,count_name,count_vec,i)
    perplexity[i]=perp
    W_lda = pd.concat([pd.DataFrame(W,index=df["PMID"],columns = ["col1", "col2"]),df_lda], axis=1)
    W_lda.to_csv("../Files/Figure4/Table{}_rs1.csv".format(i+9))
    df_period_topic.to_csv("../Files/Figure4/Table{}_rs1.csv".format(i+20))


df_perplexity = pd.DataFrame(perplexity.values(),index=perplexity.keys())
df_perplexity.to_csv("../Files/Figure4/Table41_rs1.csv")

#5 LDA (with TF-IDF) and n_topic optimization
perplexity_tfidf = {}
for i in range(10,21):
    perp, df_lda, df_period_topic = lda_create(df,tfidf_name,tfidf_vec,i)
    perplexity_tfidf[i]=perp
    W_lda_tfidf = pd.concat([pd.DataFrame(W_tfidf,index=df["PMID"],columns = ["col1", "col2"]),df_lda], axis=1)
    W_lda_tfidf.to_csv("../Files/Figure4/Table{}_rs1.csv".format(i+32))
    df_period_topic.to_csv("../Files/Figure4/Table{}_rs1.csv".format(i+43))


df_perplexity_tfidf = pd.DataFrame(perplexity_tfidf.values(),index=perplexity_tfidf.keys())
df_perplexity_tfidf.to_csv("../Files/Figure4/Table64_rs1.csv")